In [68]:
using LowLevelFEM

In [69]:
gmsh.initialize()

In [70]:
function cube_mesh(; lx=1.0, ly=1.0, lz=1.0, n=10, dx=lx / n, dy=ly / n, dz=lz / n)

    gmsh.option.setNumber("General.Verbosity", 0)

    # --------------------------------------------------
    # Geometry
    # --------------------------------------------------
    # Box: origin (0,0,0), size l x l x l
    box = gmsh.model.occ.addBox(0.0, 0.0, 0.0, lx, ly, lz)

    gmsh.model.occ.synchronize()
    #return

    # --------------------------------------------------
    # Physical groups
    # --------------------------------------------------
    gmsh.model.addPhysicalGroup(2, [1], -1, "left")
    gmsh.model.addPhysicalGroup(2, [2], -1, "right")
    gmsh.model.addPhysicalGroup(2, [6], -1, "front")
    gmsh.model.addPhysicalGroup(2, [5], -1, "rear")
    gmsh.model.addPhysicalGroup(2, [3], -1, "bottom")
    gmsh.model.addPhysicalGroup(2, [4], -1, "top")

    gmsh.model.addPhysicalGroup(3, [1], -1, "volume")

    # --------------------------------------------------
    # Mesh settings (structured hex mesh)
    # --------------------------------------------------
    gmsh.option.setNumber("Mesh.Algorithm3D", 1)      # Delaunay for recombination
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 1)

    gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

    gmsh.model.mesh.setTransfiniteAutomatic()

    gmsh.option.setNumber("Mesh.ElementOrder", 1)

    # Characteristic length control
    lc = min(dx, dy, dz)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)

    # --------------------------------------------------
    # Generate mesh
    # --------------------------------------------------
    gmsh.model.mesh.generate(3)

    # --------------------------------------------------
    # Save
    # --------------------------------------------------
    #gmsh.write("cube.msh")

    #gmsh.finalize()

    return nothing
end


cube_mesh (generic function with 1 method)

In [71]:
cube_mesh()

In [72]:
mat = Material("volume")
prob = Problem([mat]);

In [73]:
ubcx = ScalarField(prob, "left", (x, y, z, t) -> t, steps=100, tmax=1)

elementwise ScalarField
[[0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0], [0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0], [0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0], [0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0], [0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.010101010101010102 … 0.989898

In [74]:
K = stiffnessMatrix(prob)
M = massMatrix(prob);

In [75]:
load = BoundaryCondition("right", fx=0)
supp = BoundaryCondition("left", ux=ubcx);

In [76]:
f = loadVector(prob, [load]);

In [77]:
Tmin = smallestPeriodTime(K, M, support=[supp])

7.370207795921925e-8

In [78]:
u0 = VectorField(prob, "volume", [0, 0, 0])
v0 = VectorField(prob, "volume", [0, 0, 0]);

In [79]:
u, v = CDM(K, M, f, u0, v0, 100, Tmin / π, support=[supp])

(VectorField(Matrix{Float64}[], [0.0 0.010101010101010102 … 0.98989898989899 1.0; 0.0 0.0 … 0.014588505392214911 0.002115818091323827; … ; 0.0 0.0 … -0.0038370376815526604 0.0008056990771466823; 0.0 0.0 … -0.0038370376815505198 0.0008056990771501873], [0.0, 2.346010004670795e-8, 4.69202000934159e-8, 7.038030014012384e-8, 9.38404001868318e-8, 1.1730050023353975e-7, 1.407606002802477e-7, 1.6422070032695567e-7, 1.8768080037366362e-7, 2.1114090042037158e-7  …  2.1114090042037126e-6, 2.1348691042504205e-6, 2.1583292042971284e-6, 2.1817893043438363e-6, 2.205249404390544e-6, 2.228709504437252e-6, 2.25216960448396e-6, 2.275629704530668e-6, 2.2990898045773758e-6, 2.3225499046240837e-6], Int64[], 100, :v3D, Problem("", :Solid, 3, 3, Material[Material("volume", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 512, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))), VectorField(Matrix{Floa

In [80]:
showDoFResults(u, name="u");

In [81]:
s = solveStress(u);

In [82]:
showStressResults(s, name="σ", smooth=true);

In [83]:
openPostProcessor()

In [84]:
gmsh.finalize()